# Chapter 11: Motor control and reinforcement learning

### Load needed modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 11.3. Formalization of reinforcement learning
### 11.3.1. The environmental setting of a Markov decision process

To introduce some fundamental concepts in reinforcement learning, we are going to use the example situation of navigating a 1-dimensional maze in order to maximize the reward obtained in one of its ends.

The main two are the environmental functions of:
- the transition function τ (s, a), which defines how the environment changes from one state to another when an agent takes an action
- the reward function ρ(s) defines the immediate numerical feedback an agent receives after taking an action in a given state

There are also several helper functions:
- calculation of the policy from a value function
- function idx(a) to transforms the action representation a ∈ {−1, 1} to the corresponding indices idx ∈ {0, 1}, which we need for the specific implementation of the actions in this example.

In [ ]:
## Reinforcement learning in 1d maze
import numpy as np
import matplotlib.pyplot as plt

def tau(s,a):
    if s==0 or s==4:
        return(s)
    else:
        return(s+a)
def rho(s,a):
    return(s==1 and a==-1)+2*(s==3 and a==1)
def calc_policy(Q):
    policy=np.zeros(5)
    for s in range(0,5):
        action_idx=np.argmax(Q[s,:])
        policy[s]=2*action_idx-1
        policy[0]= policy[4]=0
    return policy.astype(int)

def idx(a):
    return(int((a+1)/2))

The expected reward (return) cannot increase without bounds. Thus, we keep the return finite by using a discounted return in which an agent values immediate reward more than a reward to be obtained far in the future. To capture this we define a **discount factor** 0 < *γ* < 1.

In [ ]:
# Discount factor
gamma=0.5

### 11.3.2. Model-based reinforcement learning

**Bellman equation**

Broadly speaking, the Bellman equation describes the relationship between the value of a state and the values of its possible successor states that an agent can take.

It expresses the value of a state *s* in terms of the immediate reward and the expected future value of the next state.

The Bellman equation itself is not an action-value function. Instead, it is a general recursive equation used to describe the value of a state or state-action pair in terms of future rewards. The Bellman equation can be applied to both:

- State-value function 𝑉(𝑠) → The expected return when starting from state 𝑠 and following a policy.
- Action-value function (Q-function) *Q(s,a)* → The expected return when starting from state 𝑠, taking action 𝑎, and following a policy.

It serves as the foundation for computing optimal strategies in RL by breaking down complex decision-making problems into simpler recursive relationships.

**Reward vector**

In [ ]:
print('--> Analytic solution for optimal policy')
# Defining reward vector R
i=0; R=np.zeros(10)
for s in range(0,5):
    for a in range(-1,2,2):
        R[i]=rho(s,a)
        i+=1

**Transition matrix**

The transition matrix (Fig 11.6) is the mathematical representation of how an environment changes states when an agent takes actions (based on states, actions, and rewards).

It depends on the policy, so we need to choose one. We chose the one specified on the left in Fig. 11.6 where the agent would move to the left in state s = 1 and to the right in states s = 2 and s = 3.

In [ ]:
# Defining transition matrix
T = np.zeros([10,10])
T[0,0]=1; T[1,1]=1; T[2,0]=1; T[3,5]=1; T[4,2]=1
T[5,7]=1; T[6,5]=1; T[7,9]=1; T[8,7]=1; T[9,9]=1

**Q-function**

The Q-function is a value function that represents the expected cumulative reward an agent can obtain by taking an action 𝑎 in a state 𝑠 and following an optimal policy thereafter.

In the code below, following the 1D maze example, the first row shows the values for left movements in each state and the second row shows the values for a right movement in each state.

In [ ]:
# Calculate Q-function
Q=np.linalg.inv(np.eye(10)-gamma*T) @ np.transpose(R)
Q=np.reshape(Q,[5,2]); Q[4,0]=0

**Policy**

A policy (𝜋) is a strategy or rule that an agent follows to decide which action to take in a given state. It defines the agent’s behavior at every step of interaction with the environment.

In [ ]:
# Defining policy based on our
policy=calc_policy(Q)

In [ ]:
print('Q values: \n', np.transpose(Q))
print('policy: \n', np.transpose(policy))

Below we simply store the optimal Q-values for the optimal policy in this analytical solution, for later comparison with other methods

In [ ]:
Qana=Q

**Dynamic programming**

Dynamic programming provides algorithms to compute the value function and optimal policy by iteratively solving the Bellman equation.
It is used to for value function iteration and policy iteration and evaluation.

Dynamic programming is useful when: 
- The environment’s transition probabilities and rewards are known.
- The state space is small (DP is infeasible for large-scale problems due to high memory and computation costs).
- Used as a baseline for approximate solutions like Q-learning or Deep RL.

In [ ]:
print('---> Dynamic Programming')

Q=np.zeros([5,2])
for iter in range(3):
    for s in range(0,5):
        for a in range(-1,2,2):
            act=int(policy[tau(s,a)])
            Q[s, idx(a)]=rho(s,a)+gamma*Q[tau(s,a), idx(act)]
            
print('Q values: \n', np.transpose(Q))
print('policy: \n', np.transpose(policy))

**Policy iteration**

Policy iteration is a way of finding the policy to maximize the return.

It consists in starting with an arbitrary policy and using the corresponding value function to define a new policy which is given by taking the actions from each state that will give us the best next return value.

For the new policy, we can calculate the corresponding Q-function and then use this Q-function to improve the policy again (iterative process).

In [ ]:
print('---> Policy iteration')

Q=np.zeros([5,2])
policy=calc_policy(Q)
for iter in range(3):
    for s in range(0,5):
        for a in range(-1,2,2):
            act = int(policy[tau(s,a)])
            Q[s,idx(a)]=rho(s,a)+gamma*Q[tau(s,a),idx(act)]
    policy = calc_policy(Q)
    
print('Q values: \n', np.transpose(Q))
print('policy: \n', np.transpose(policy))

**Q-iteration**

Q-iteration is used in model-based RL to iteratively compute the Q-value function by leveraging a learned or known environment model.

It follows the principle of value iteration, but instead of updating the state-value function, it updates the action-value function. Given a learned transition and reward function, the Q-values are updated iteratively.

In [ ]:
print('---> Q-iteration')

Q_new=np.zeros([5,2])
Q=np.zeros([5,2])
for iter in range(3):
    for s in range(0,5):
        for a in range(-1,2,2):
            maxValue = np.maximum(Q[tau(s,a),0],Q[tau(s,a),1])
            Q_new[s,idx(a)]=rho(s,a)+gamma*maxValue
    Q=np.copy(Q_new)
    
policy=calc_policy(Q)
print('Q values: \n', np.transpose(Q))
print('policy: \n', np.transpose(policy))

### 11.3.3. Model-free reinforcement learning

In model-free RL, we combine sampling with RL by exploring the environment.

Similarly to model-based RL, we want to minimize the difference between the left- and the right-hand sides of the Bellman equation.

But we cannot calculate the right-hand side because we don't know the transition or the reward functions (which we do know in model-based RL).

**SARSA**

Therefore, we use the SARSA (state->action->reward->state->action) algorithm.

SARSA consists in taking a step based on our policy, and observe a reward ($r_{i+1}$) and a next state ($s_{i+1}$).

Since this is only one step and sample, we should use this update of the value function only with a small learning rate (*α*).

In [ ]:
print('---> SARSA')

Q=np.zeros([5,2])
error = []
alpha = 1
for trial in range(200):
    policy=calc_policy(Q)
    s=2
    for t in range(0,5):
        a=policy[s]
        if np.random.rand()<0.1: a=-a #epsilon greedy
        a2=idx(policy[tau(s,a)])
        TD=rho(s,a)+gamma*Q[tau(s,a),a2]-Q[s,idx(a)]
        Q[s, idx(a)]=Q[s, idx(a)]+alpha*TD
        s=tau(s,a)
    error.append(np.sum(np.sum(np.abs(np.subtract(Q,Qana)))))

print('Q values: \n', np.transpose(Q))
print('policy: \n', np.transpose(policy))
plt.figure(); plt.plot(error)
plt.xlabel('iteration'); plt.ylabel('error (Q - Qana)')

**Q-learning**

Q-learning is a model-free RL algorithm that learns the optimal q-function without requiring prior knowledge of the environment's transition probabilities.

It is an off-policy, value-based method that uses Q-values (action-value function) to estimate the expected cumulative reward for taking an action in a given state. As a result, it updates Q-values iteratively using the Bellman equation.

In [ ]:
print('--->Q-Learning:')

Q=np.zeros([5,2])
alpha=1
error=[]
for trial in range(200):
    policy=calc_policy(Q)
    s=2
    for t in range(0,5):
        a=policy[s]
        if np.random.rand()<0.2: a =-a #epsilon greedy
        TD=rho(s,a)+gamma*np.maximum(Q[tau(s,a),0],Q[tau(s,a),1])-Q[s,idx(a)]
        Q[s,idx(a)]=Q[s,idx(a)]+alpha*TD
        Q[0]=0; Q[4]=0
        s=tau(s,a)
    error.append(np.sum(np.sum(np.abs(np.subtract(Q,Qana)))))
    
print('Q values: \n', np.transpose(Q))
print('policy: \n', np.transpose(policy))
plt.plot(error,'r')
plt.xlabel('iteration'); plt.ylabel('error (Q - Qana)')

**TD(λ)**

TD(λ), or temporal difference with eligibility traces, is controlled by a trace decay parameter 𝜆, which allows for both short-term and long-term credit assignment.

An eligibility trace keeps track of how much credit each state (or state-action pair) should receive for future rewards. It works like a memory that assigns weight to past states, depending on how recent they were visited.

The value function update of TD(λ) depends on λ:
- **λ**=0: Equivalent to TD(0) (only updates based on the most recent step).
- **λ**=1: Equivalent to Monte Carlo (updates values using full-episode returns).
- 0<**λ**<1: A trade-off, balancing short-term and long-term learning.

In [ ]:
print('---> TD(lambda) for Q-learning')

Q=np.zeros([5,2])
alpha=1
error=[]
eligibility=np.zeros([5,2])
lam=0.7

for trial in range(200):
    policy=calc_policy(Q)
    s=2
    for t in range(0,5):
        a=policy[s]
        if np.random.rand()<0.1: a=-a #epsilon greedy
        TD=rho(s,a)+gamma * np.maximum(Q[tau(s,a),0], Q[tau(s,a),1])-Q[s,idx(a)]
        eligibility*=gamma*lam
        eligibility[s,idx(a)]=1
        for si in range(1,4):
            for ai in range(2):
                Q[si, ai]=Q[si, ai]+alpha*TD*eligibility[si,ai]
        Q[0]=0; Q[4]=0;
        s=tau(s,a)
        error.append(np.sum(np.sum(np.abs(np.subtract(Q,Qana)))))
        
print('Q values: \n', np.transpose(Q))
print('policy: \n', np.transpose(policy))
plt.plot(error, 'r');
plt.xlabel('iteration'); plt.ylabel('error')    

## 11.4. Deep reinforcement learning
### 11.4.1. Value-function approximation with ANN

Given that the "cartoonish" way of approximating a function with tabular methods is unfeasible in real-life applications, we can make use of more complex function approximations.

There are simple methods, such as linear regression, that work fine as a baseline method, since they are also tractable analytically. 

However, since many real-world applications are highly non-linear, ANN become more useful.

**Neurally-fitted Q-iteration (NFQ)** is an example one example of the algorithms for value-function approximation with ANN (Fig 11.11 B). It consists in training a network that provides the Q-values for all the possible actions.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras import models, layers, optimizers

def tau(s,a):
    if (s[0] and s[4]) == 0:
        s=np.roll(s,a)
    return s
def rho(s):
        return ((s[0]==1)+2*(s[4]==1))
def terminal_state(s):
        return (s[0]==1 or s[4]==1)

gamma=0.5
invT=1

Here, we specify a **small dense network** with 5 inputs for the state vector, 10 hidden nodes, and 2 output nodes, each representing the Q-value for each possible action, that of going left or right.

In [ ]:
# the network
inputs = layers.Input(shape=(5,))
h = layers.Dense(10, activation='relu')(inputs)
outputs = layers.Dense(2, activation='linear')(h)

model = models.Model(inputs=inputs, outputs=outputs)
RMSprop = optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='mse', optimizer=RMSprop)

From the current state we use the network to predict the corresponding *Q-value* and then move one step ahead to calculate the target for the gradient learning as *r + Q(next_s)*. The network is then updated right away.

In [ ]:
for trial in range(400):
    s = np.array([0,0,1,0,0])
    for t in range(0,5):
        if terminal_state(s): break
        if trial > 30 and invT > 0.1: invT -= 0.001
        prediction=model.predict(s.reshape(1,5), steps=1, verbose=0)
        aidx=np.argmax(prediction)
        if np.random.rand() < invT: aidx=1-aidx
        a=2*aidx-1
        next_s = tau(s,a)
        if terminal_state(next_s):
            y = rho(next_s)
        else:
            y = gamma*np.max(model.predict(next_s.reshape(1,5), steps=1, verbose=0))
            prediction[0, aidx]=y
            model.fit(s.reshape(1,5), prediction, epochs=1, verbose=0)
            s = np.copy(next_s)

After the exploration we can evaluate the final policy and value functions:

In [ ]:
for trial in range(400):
    s = np.array([0,0,1,0,0])
    for t in range(0,5):
        if terminal_state(s): break
        if trial > 30 and invT > 0.1: invT -= 0.001
        prediction=model.predict(s.reshape(1,5), steps=1, verbose=0)
        aidx=np.argmax(prediction)
        if np.random.rand() < invT: aidx=1-aidx
        a=2*aidx-1
        next_s = tau(s,a)
        if terminal_state(next_s):
            y = rho(next_s)
        else:
            y = gamma*np.max(model.predict(next_s.reshape(1,5), steps=1, verbose=0))
            prediction[0, aidx]=y
            model.fit(s.reshape(1,5), prediction, epochs=1, verbose=0)
            s = np.copy(next_s)

### 11.4.7. Neural implementations of TD learning

TD learning updates value estimates based on the difference between consecutive time steps, rather than waiting for the final reward at the end of an episode. The key idea is to learn from incomplete episodes by bootstrapping (updating estimates using other estimates). Given this incremental feature of updating its values at every step, it becomes more sample-efficient than e.g., Monte Carlo methods.

TD learning provides a computational model of reward-based learning that aligns to an extent with biological reinforcement mechanisms in behavior (e.g., Rescorla-Wagner) and the brain (e.g., Schulz et al., 1997).

The dopamine system plays a crucial role in encoding prediction errors, while the basal ganglia and cortex help store and update learned values, enabling adaptive behavior.

In [ ]:
gamma=0
w=np.zeros(10)
Z=np.zeros((100,6)); rhatREC=np.zeros(100)
x=np.random.rand(10,6)
r=np.array([0,0,1,0,0.5,0])

for trial in range(100):
    V=0
    for t in range(1,6):
        Vlast=V
        V=w@x[:,t]
        rhat=r[t-1]+gamma*V-Vlast
        w=w+0.1*rhat*x[:,t-1]
        Z[trial,t-1]=Vlast
        rhatREC[trial]=rhatREC[trial]+rhat
plt.figure(); plt.plot(Z[-1,:])
plt.figure(); plt.plot(rhatREC/5)

TD learning has the ability of **transferring the reward response to earlier predictive cues** (conditioned stimuli), and such characteristic neural responses where discovered by Schulz et al. (1997) in dopaminergic neurons in some areas of the midbrain called the Ventral Tegmental Area (VTA) and the Substantia Nigra (SN).

Such reward-response transfer to the conditioned stimulus is simulated in the code below.

In [ ]:
gamma=1
w=np.zeros(20); Z=np.zeros((30,21))

for trial in range(30):
    x=np.zeros(20); V=0;
    for t in range(1,20):
        xlast=np.copy(x)
        Vlast=V
        x[t]=x[t-1] # shift visual memory
        x[t-1]=0
        if t==2: x[t]=1 # vis onset
        print(t,x) 
        V=w@x
        rhat=gamma*V-Vlast
        w=w+1*rhat*xlast
        Z[trial,t]=rhat
    rhat=1-V # last time gets reward
    w=w+1*rhat*x
    Z[trial, t+1]=rhat
plt.imshow(Z, cmap='binary')

## Homework
Due Monday, Mar. 31, 13.00. Send to jperez@bcbl.eu in either .doc, .txt, or just by inserting it in the markdown cell below and sending me a copy of this notebook
1. Do you see the fundamentals of reinforcement learning (e.g., state, reward, actions) as more based on behavioral or neurobiological foundations? Or both? Develop a bit your response ;)
2. What elements for learning do you see that temporal difference includes? Do you see those elements leaning more towards the reward-driven learning or also as elements for predictive processing/learning? It can be just your impression and thoughts from a first glance at the chapter! 
3. Can you speculate on a potential model that uses reinforcement learning to explore/simulate a language process (e.g., vocabulary learning, speech tracking, morphological processing?)